In [51]:
!pip install nilearn==0.9.2

# !pip install keras_cv


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import os
from sklearn.linear_model import LinearRegression
import tensorflow_datasets as tfds
import tensorflow as tf
from keras import datasets, Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
import numpy as np
from sklearn.metrics import confusion_matrix
# from tensorflow.keras.layers.experimental.preprocessing import Resizing
# import keras_cv
import keras
import datetime

# %load_ext tensorboard

# Load the data

In [53]:
########## LOADING CIFAR DATASET #############

# cifar, cifar_info = tfds.load("cifar100", as_supervised=True, with_info =True)
(train_fab, train_y), (test_fab, test_y)= tf.keras.datasets.cifar100.load_data(label_mode='fine')

# Preprocess data

In [56]:
############# PREPROCESS FUNCTION #################

def data_pip(cifar10, dataset_type, batch_size):

  # data = tf.data.Dataset.from_tensor_slices(data)
  cifar10 = cifar10.map(lambda img, target: (tf.cast(img, tf.float32), target))
  
  #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
  cifar10 = cifar10.map(lambda img, target: ((img/128.)-1., target))
  #create one-hot targets
  cifar10 = cifar10.map(lambda img, target: (img, tf.one_hot(target, depth=100)))
  #cache this progress in memory, as there is no need to redo it; it is deterministic after all
  # cifar10 = cifar10.cache()
  #shuffle, batch, prefetch
  # cifar10 = cifar10.shuffle(1000)
  cifar10 = cifar10.map(lambda img, target: (tf.expand_dims(img, 0), target))
  cifar10 = cifar10.batch(32)
  return cifar10

In [58]:
########### PREPROCESS ################

issa = tf.data.Dataset.from_tensor_slices((train_fab, train_y))

cifar_data = data_pip(issa, 'cifar', 32)
print(cifar_data)

<MapDataset element_spec=(TensorSpec(shape=(1, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(1, 100), dtype=tf.float32, name=None))>


# Visualization

In [59]:
############ VISUALIZATION ################

# tfds.show_examples(cifar_data, cifar_info)

# CNN Model

In [62]:
############# NO OPTIMIZERS ####################

def create_CNN_model():

    model = Sequential()
    model.add(Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
    model.add(Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
    model.add(Conv2D(input_shape=(32, 32, 3), kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))

    model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
    model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
    model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(100, activation='softmax'))


    model.compile(loss='CategoricalCrossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    return model



In [67]:
########## CREATE MODEL ###########

cnnn_model = create_CNN_model()

cnnn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 16, 16, 32)        416       
                                                                 
 conv2d_25 (Conv2D)          (None, 8, 8, 32)          4128      
                                                                 
 conv2d_26 (Conv2D)          (None, 4, 4, 32)          4128      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 4, 32)         0         
 2D)                                                             
                                                                 
 conv2d_27 (Conv2D)          (None, 2, 2, 64)          8256      
                                                                 
 conv2d_28 (Conv2D)          (None, 1, 1, 64)          16448     
                                                      

# ResNet50 Model

In [64]:
############## ResNet50 MODEL  #############

resnet_model = tf.keras.applications.resnet.ResNet50(
       input_shape=(32,32,3), #.output_shapes()[0],
       include_top=True,
       weights=None)

resnet_model.compile(
       optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), #optimizer=tf.keras.optimizers.SGD(lr=0.001),
      loss='sparse_categorical_crossentropy', #loss='binary_crossentropy',
       metrics=['accuracy'])


# Train & Test Model
weights and tensorboard logging

In [70]:
#set tensorboard 
# current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# logging_callback = tf.keras.callbacks.TensorBoard(log_dir=f"./logs/CNN_model/{current_time}")

# Train the model
cnnn_model.fit(cifar_data, batch_size=64, epochs=5) #callbacks=[logging_callback]) #batch_size=64, 

# Test Step
test_loss, test_acc = cnnn_model.evaluate(cifar_data)
print("test accuracy: ", test_acc)

# save the weights
# cnn_model.save_weights('./checkpoints/my_checkpoint')

# logging with tensorboard
# %tensorboard --logdir="logs/CNN_model"

Epoch 1/5
50000/50000 [==============================] - 297s 6ms/step - loss: 4.4117 - accuracy: 0.0285
Epoch 2/5
50000/50000 [==============================] - 282s 6ms/step - loss: 4.6087 - accuracy: 0.0093
Epoch 3/5
50000/50000 [==============================] - 279s 6ms/step - loss: 4.6087 - accuracy: 0.0090
Epoch 4/5
50000/50000 [==============================] - 289s 6ms/step - loss: 4.6087 - accuracy: 0.0090
Epoch 5/5
50000/50000 [==============================] - 155s 3ms/step - loss: 4.6067 - accuracy: 0.0100
test accuracy:  0.009999999776482582


In [ ]:
########### LOAD OLD WEIGHTS #########
'''
# Create a new model instance
new_model = create_CNN_model()

# Restore the weights
new_model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = new_model.evaluate(x_test_cifar, y_test_cifar)
print("test accuracy: ", acc)'''


# Confusion Matrix